In [1]:
import torch

# Load the tensor
indices = torch.load('indices.pt')



C:\Users\tharu\AppData\Local\Temp\ipykernel_10288\569774741.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  indices = torch.load('indices.pt')


In [ ]:
indices_np = indices.numpy()  # Convert PyTorch tensor to NumPy array



In [49]:
indices

array([100, 329, 122,  67, 288, 126, 128, 140, 460, 372])

uint 16 for k=512

In [125]:
import numpy as np
import reedsolo

# Initialize RS codec
r = 4  # Redundancy (4 symbols per block)
n = 255  # Total symbols per block
k = n - r  # Data symbols per block
rs = reedsolo.RSCodec(r)

# Simulated data
original_size = 504 # Original data size
indices = np.random.randint(0, 512, size=(original_size,), dtype=np.uint16)

# Calculate padding
padding_length = (k - (len(indices) % k)) % k
padded_indices = np.pad(indices, (0, padding_length), mode='constant', constant_values=0)
indices_bytes = padded_indices.tobytes()

# Ensure that padded data length aligns with expected block size
assert len(indices_bytes) % k == 0, "Padded data size is not aligned with the block size."

# Encode the data
encoded_data = rs.encode(indices_bytes)

# Simulate noise (add random bit errors)
def add_noise(data, num_errors):
    noisy_data = bytearray(data)
    for _ in range(num_errors):
        byte_index = np.random.randint(len(noisy_data))
        bit_index = np.random.randint(8)
        noisy_data[byte_index] ^= (1 << bit_index)  # Flip a random bit
    return bytes(noisy_data)

noisy_data = add_noise(encoded_data, num_errors=20)  # Add errors across blocks

# Process data block by block
def decode_blocks(encoded_data, block_size, data_size, redundancy):
    decoded_result = bytearray()
    num_blocks = len(encoded_data) // block_size

    print(f"Decoding {num_blocks} blocks of size {block_size} bytes each.")

    for i in range(num_blocks):
        start = i * block_size
        end = start + block_size
        block = encoded_data[start:end]

        try:
            # Attempt to decode the block
            decoded_block = rs.decode(block)
            if isinstance(decoded_block, tuple):  # Handle tuple output
                decoded_block = decoded_block[0]
            decoded_result.extend(decoded_block[:data_size])  # Append only data symbols
        except reedsolo.ReedSolomonError:
            # Decoding failed, append raw noisy data for this block
            print(f"Block {i + 1} could not be decoded. Retaining noisy data.")
            decoded_result.extend(block[:data_size])  # Append raw data symbols (exclude redundancy)

    return decoded_result

# Decode the noisy data
decoded_data = decode_blocks(noisy_data, block_size=n, data_size=k, redundancy=r)

# Remove padding
decoded_data = decoded_data[:original_size * 2]  # Trim to original data size (bytes)
decoded_indices = np.frombuffer(decoded_data, dtype=np.uint16)

# Validate shapes
print("Decoded Indices (Best Effort):")
print(decoded_indices)

print("\nOriginal Indices:")
print(indices)

print(f"\nOriginal shape: {indices.shape}, Decoded shape: {decoded_indices.shape}")


Decoding 6 blocks of size 255 bytes each.
Block 5 could not be decoded. Retaining noisy data.
Decoded Indices (Best Effort):
[  394   290   165   493    23   129   121   285   113   427   393    23
    77   229   419   188    82   265   297   125   450   283   457   128
   455   213   424   295   302   286    29   119   402   315   436   107
   429   378   243    69   229   280   316   275    84   476   238   283
   113    20   466   287   453   372    44   256   255   377    51   185
   393   392   373   493    38   158     7    50   365   414    16   246
   170   197   336   195   403   509   130   298   128   375   135   450
   381   406   283    28    19   237   472   433   167   286   131   421
   291    91   188   281   209   226   412   236    21   457   484   350
   108   263   179    64   101    87   374    35   388   480   135    13
   329   271    13   116   132   172   330   188   102   119   387   240
   466   212   415   501   253   480    21   250   149   331   210   423

uint 8

In [148]:
import numpy as np
import reedsolo

# Initialize RS codec
r = 120  # Redundancy (4 symbols per block)
n = 255  # Total symbols per block
k = n - r  # Data symbols per block
rs = reedsolo.RSCodec(r)

# Simulated data
original_size = 503  # Original data size
indices = np.random.randint(0, 256, size=(original_size,), dtype=np.uint8)  # uint8 indices

# Calculate padding
padding_length = (k - (len(indices) % k)) % k  # Padding to align data size to block size
padded_indices = np.pad(indices, (0, padding_length), mode='constant', constant_values=0)
indices_bytes = padded_indices.tobytes()

# Ensure that padded data length aligns with expected block size
assert len(indices_bytes) % k == 0, "Padded data size is not aligned with the block size."

# Encode the data
encoded_data = rs.encode(indices_bytes)

# # Simulate noise (add random bit errors)
# def add_noise(data, num_errors):
#     noisy_data = bytearray(data)
#     for _ in range(num_errors):
#         byte_index = np.random.randint(len(noisy_data))
#         bit_index = np.random.randint(8)
#         noisy_data[byte_index] ^= (1 << bit_index)  # Flip a random bit
#     return bytes(noisy_data)

# noisy_data = add_noise(encoded_data, num_errors=20)  # Add errors across blocks

# Simulate noise (add random bit errors based on error ratio)
def add_noise_with_ratio(data, error_ratio):
    """
    Add noise to the encoded data by flipping bits based on the error ratio.

    Args:
        data (bytes): Encoded data.
        error_ratio (float): Fraction of bits to flip (e.g., 0.01 for 1% errors).

    Returns:
        bytes: Noisy data.
    """
    noisy_data = bytearray(data)
    total_bits = len(noisy_data) * 8  # Total number of bits in the data
    num_errors = int(total_bits * error_ratio)  # Calculate the number of bits to flip

    print(f"Introducing {num_errors} errors out of {total_bits} total bits (ratio = {error_ratio}).")

    for _ in range(num_errors):
        byte_index = np.random.randint(len(noisy_data))  # Random byte index
        bit_index = np.random.randint(8)  # Random bit within the byte
        noisy_data[byte_index] ^= (1 << bit_index)  # Flip the selected bit

    return bytes(noisy_data)

# Introduce noise with a given error ratio
error_ratio = 0.025  # Flip 1% of the total bits
noisy_data = add_noise_with_ratio(encoded_data, error_ratio)

# Process data block by block
def decode_blocks(encoded_data, block_size, data_size, redundancy):
    decoded_result = bytearray()
    num_blocks = len(encoded_data) // block_size

    print(f"Decoding {num_blocks} blocks of size {block_size} bytes each.")

    for i in range(num_blocks):
        start = i * block_size
        end = start + block_size
        block = encoded_data[start:end]

        try:
            # Attempt to decode the block
            decoded_block = rs.decode(block)
            if isinstance(decoded_block, tuple):  # Handle tuple output
                decoded_block = decoded_block[0]
            decoded_result.extend(decoded_block[:data_size])  # Append only data symbols
        except reedsolo.ReedSolomonError:
            # Decoding failed, append raw noisy data for this block
            print(f"Block {i + 1} could not be decoded. Retaining noisy data.")
            decoded_result.extend(block[:data_size])  # Append raw data symbols (exclude redundancy)

    return decoded_result

# Decode the noisy data
decoded_data = decode_blocks(noisy_data, block_size=n, data_size=k, redundancy=r)

# Remove padding
decoded_data = decoded_data[:original_size]  # Trim to original data size (bytes)
decoded_indices = np.frombuffer(decoded_data, dtype=np.uint8)

# Validate shapes
print("Decoded Indices (Best Effort):")
print(decoded_indices)

print("\nOriginal Indices:")
print(indices)

print(f"\nOriginal shape: {indices.shape}, Decoded shape: {decoded_indices.shape}")


Introducing 204 errors out of 8160 total bits (ratio = 0.025).
Decoding 4 blocks of size 255 bytes each.
Decoded Indices (Best Effort):
[196 240   4 104 239 171 149 214 157 101 177  85 208 157 171 129 124 229
  42 106 165 164 245 167 247 244  54  18  43 102 196 242  82  79  23 111
   9   2  21  40 186 120 214  27 105  58 110 121 193  60 199  87 189 174
 218 209 186 186 134  99 188 207  93 186  36  55 138 113 201 112 244 129
  40 235 112 207 183  62 135  62 225 166 120 193 138  28 167 131   0 132
  62 157  15  56 135  91 125 117 167 239   6 199 218 240 117 104  50 170
 157 235 250  72 230  10 218 231 152 143 197  52 129 189 168  15  91  31
 230 231 240 120  29 227  18 125 227  53  81 150  75  16  50 128   9 144
  54 177 125 176 231  58 174  38 147 196  84 165 106 203 136 126 200 133
 150  35 210 200 140  17 214  62 234 178 184  89 158  71  55  88 159  43
  36 229 185  90 175  59 218 215 165  57 198 181   1  38 121 185  38  81
 215  70 233 229 231 250  55  97 202 150 161  76   9   5 159 

Transmitter

Transmitter for swin

In [188]:
import torch
import numpy as np
import reedsolo


#indices, _ = encode_image_with_nd_codebook(net, codebook_path, test_image_path, config, device, chunk_size)

indices = torch.load('indices_4d_256k.pt')   # Getting indices from that
indices_np = indices.numpy()  # Convert PyTorch tensor to NumPy array  - This is stored in int64

indices_np = indices_np.astype(np.uint8)  # Convert to uint8

#indices = np.fromfile('indices_4d_256k_rayleigh_img22.bin', dtype=np.uint8)


# Initialize RS codec
r = 127  # Redundancy (4 symbols per block)
n = 255  # Total symbols per block  , Do not change this
data_symbols = n - r  # Data symbols per block
rs = reedsolo.RSCodec(r)

# Transmitter
def encode_blocks(indices, output_file):
    # Calculate padding
    padding_length = (data_symbols - (len(indices) % data_symbols)) % data_symbols  # Padding to align data size to block size
    padded_indices = np.pad(indices, (0, padding_length), mode='constant', constant_values=0)
    indices_bytes = padded_indices.tobytes()

    # Ensure that padded data length aligns with expected block size
    assert len(indices_bytes) % data_symbols == 0, "Padded data size is not aligned with the block size."

    # Encode the data
    encoded_data = rs.encode(indices_bytes)

    # Save encoded data to a .bin file
    with open(output_file, "wb") as f:
        f.write(encoded_data)

    print(f"Encoded data saved to {output_file}")
    return len(indices), len(padded_indices), encoded_data

k = 256
chunk_size = 4
img_no = '22'
# Transmit the encoded data
original_size, padded_size, encoded_data = encode_blocks(indices_np, f"indices_ec_{chunk_size}d_{k}k_img{img_no}.bin")




Encoded data saved to indices_ec_4d_256k_img22.bin


C:\Users\tharu\AppData\Local\Temp\ipykernel_29956\2663710079.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  indices = torch.load('indices_4d_256k.pt')   # Getting indic

Simulating noise

In [189]:
import numpy as np

def add_noise_to_bin_file(input_file, output_file, error_ratio):
    """
    Adds noise to an encoded .bin file by flipping a fraction of bits.
    
    Args:
        input_file (str): Path to the input encoded .bin file.
        output_file (str): Path to save the noisy .bin file.
        error_ratio (float): Fraction of bits to flip (e.g., 0.01 for 1% of bits).
    """
    # Read the encoded data from the input file
    with open(input_file, "rb") as f:
        encoded_data = bytearray(f.read())

    # Calculate total number of bits and the number of errors to introduce
    total_bits = len(encoded_data) * 8
    num_errors = int(total_bits * error_ratio)
    print(f"Introducing {num_errors} errors out of {total_bits} total bits (ratio = {error_ratio}).")

    # Add noise by flipping random bits
    for _ in range(num_errors):
        byte_index = np.random.randint(len(encoded_data))  # Choose a random byte
        bit_index = np.random.randint(8)  # Choose a random bit within that byte
        encoded_data[byte_index] ^= (1 << bit_index)  # Flip the chosen bit

    # Save the noisy data to the output file
    with open(output_file, "wb") as f:
        f.write(encoded_data)

    print(f"Noisy data saved to {output_file}")

# Example Usage
input_file = f"indices_ec_{chunk_size}d_{k}k_img{img_no}.bin" # Path to the encoded .bin file
output_file = f"received_indices_ec_{chunk_size}d_{k}k_img{img_no}.bin"   # Path to save the noisy .bin file
error_ratio = 0.035               # Introduce 1% bit errors

add_noise_to_bin_file(input_file, output_file, error_ratio)


Introducing 6854 errors out of 195840 total bits (ratio = 0.035).
Noisy data saved to received_indices_ec_4d_256k_img22.bin


In [173]:
original_size, padded_size, len(encoded_data)

(12288, 12328, 23460)

Receiver for Swin

In [191]:
import torch
import numpy as np
import reedsolo

# Initialize RS codec
r = 127  # Redundancy (4 symbols per block)
n = 255  # Total symbols per block, do not change this
data_symbols = n - r  # Data symbols per block
rs = reedsolo.RSCodec(r)

# Original size of the data before encoding
original_size = 12288  # Know original size  (32 x H x W / (16 x 16 x 4))  4 because of 4d codebook

k = 256
chunk_size = 4
img_no = '22'

resolution = (512,768)
original_size = 32 * resolution[0] * resolution[1] // (16 * 16 * chunk_size)  # Know original size  (32 x H x W / (16 x 16 x 4))  4 because of 4d codebook


# Function to process and decode blocks from a .bin file
def decode_blocks(input_file, block_size, data_size, redundancy, original_size):
    """
    Decodes a .bin file containing noisy encoded data block by block.
    """
    decoded_result = bytearray()

    # Load the noisy encoded data from the file
    with open(input_file, "rb") as f:
        encoded_data = f.read()

    # Calculate number of blocks
    num_blocks = len(encoded_data) // block_size
    print(f"Decoding {num_blocks} blocks of size {block_size} bytes each.")

    for i in range(num_blocks):
        start = i * block_size
        end = start + block_size
        block = encoded_data[start:end]

        try:
            # Attempt to decode the block
            decoded_block = rs.decode(block)
            if isinstance(decoded_block, tuple):  # Handle tuple output
                decoded_block = decoded_block[0]
            decoded_result.extend(decoded_block[:data_size])  # Append only data symbols
        except reedsolo.ReedSolomonError:
            # Decoding failed, append raw noisy data for this block
            print(f"Block {i + 1} could not be decoded. Retaining noisy data.")
            decoded_result.extend(block[:data_size])  # Append raw data symbols (exclude redundancy)

    # Trim decoded result to match the original size
    return decoded_result[:original_size]

# Path to the noisy .bin file at the receiver
input_file = f"received_indices_ec_{chunk_size}d_{k}k_img{img_no}.bin"
output_indices_file = f"recovered_indices_{chunk_size}d_{k}k_img{img_no}.bin"

# Decode the noisy data from the .bin file
decoded_data = decode_blocks(input_file, block_size=n, data_size=data_symbols, redundancy=r, original_size=original_size)

# Convert the decoded bytearray to indices
decoded_indices = np.frombuffer(decoded_data, dtype=np.uint8)

# Save recovered indices to a binary file
with open(output_indices_file, "wb") as f:
    f.write(decoded_indices.tobytes())

print(f"Recovered indices saved to {output_indices_file}.")

recovered_indices = torch.from_numpy(decoded_indices).int()  # Convert to int32
print("Recovered Indices Tensor:")
print(recovered_indices)


Decoding 96 blocks of size 255 bytes each.
Block 5 could not be decoded. Retaining noisy data.
Block 8 could not be decoded. Retaining noisy data.
Block 11 could not be decoded. Retaining noisy data.
Block 12 could not be decoded. Retaining noisy data.
Block 13 could not be decoded. Retaining noisy data.
Block 14 could not be decoded. Retaining noisy data.
Block 17 could not be decoded. Retaining noisy data.
Block 19 could not be decoded. Retaining noisy data.
Block 20 could not be decoded. Retaining noisy data.
Block 29 could not be decoded. Retaining noisy data.
Block 30 could not be decoded. Retaining noisy data.
Block 32 could not be decoded. Retaining noisy data.
Block 34 could not be decoded. Retaining noisy data.
Block 36 could not be decoded. Retaining noisy data.
Block 42 could not be decoded. Retaining noisy data.
Block 49 could not be decoded. Retaining noisy data.
Block 55 could not be decoded. Retaining noisy data.
Block 57 could not be decoded. Retaining noisy data.
Block

In [185]:
decoded_indices

array([240, 197,  79, ...,  35,  60, 105], dtype=uint8)

In [192]:
# calculate number of similar elements between indices and decoded indices and give percentage

def calculate_similarity(indices, decoded_indices):
    num_similar = np.sum(indices == decoded_indices)
    total_elements = indices.size
    similarity = num_similar / total_elements
    return similarity, num_similar, total_elements

similarity,num_similar, total_elements = calculate_similarity(indices_np, decoded_indices)
print(f"\nSimilarity: {similarity:.2%}")
print(f"Errors: {total_elements - num_similar} out of {total_elements} elements")


Similarity: 92.02%
Errors: 980 out of 12288 elements


Add noise

In [149]:
import numpy as np

def add_noise_to_bin_file(input_file, output_file, error_ratio):
    """
    Adds noise to an encoded .bin file by flipping a fraction of bits.
    
    Args:
        input_file (str): Path to the input encoded .bin file.
        output_file (str): Path to save the noisy .bin file.
        error_ratio (float): Fraction of bits to flip (e.g., 0.01 for 1% of bits).
    """
    # Read the encoded data from the input file
    with open(input_file, "rb") as f:
        encoded_data = bytearray(f.read())

    # Calculate total number of bits and the number of errors to introduce
    total_bits = len(encoded_data) * 8
    num_errors = int(total_bits * error_ratio)
    print(f"Introducing {num_errors} errors out of {total_bits} total bits (ratio = {error_ratio}).")

    # Add noise by flipping random bits
    for _ in range(num_errors):
        byte_index = np.random.randint(len(encoded_data))  # Choose a random byte
        bit_index = np.random.randint(8)  # Choose a random bit within that byte
        encoded_data[byte_index] ^= (1 << bit_index)  # Flip the chosen bit

    # Save the noisy data to the output file
    with open(output_file, "wb") as f:
        f.write(encoded_data)

    print(f"Noisy data saved to {output_file}")

# Example Usage
input_file = "transmitted_data_4d_256k.bin"  # Path to the encoded .bin file
output_file = "noisy_data.bin"   # Path to save the noisy .bin file
error_ratio = 0.025               # Introduce 1% bit errors

add_noise_to_bin_file(input_file, output_file, error_ratio)


Introducing 2499 errors out of 99960 total bits (ratio = 0.025).
Noisy data saved to noisy_data.bin


Simulation

In [ ]:
import numpy as np
import reedsolo

# Initialize RS codec
r = 110  # Redundancy (4 symbols per block)
n = 255  # Total symbols per block
k = n - r  # Data symbols per block
rs = reedsolo.RSCodec(r)

def transmitter(indices, output_file):
    # Calculate padding
    padding_length = (k - (len(indices) % k)) % k  # Padding to align data size to block size
    padded_indices = np.pad(indices, (0, padding_length), mode='constant', constant_values=0)
    indices_bytes = padded_indices.tobytes()

    # Ensure that padded data length aligns with expected block size
    assert len(indices_bytes) % k == 0, "Padded data size is not aligned with the block size."

    # Encode the data
    encoded_data = rs.encode(indices_bytes)

    # Save encoded data to a .bin file
    with open(output_file, "wb") as f:
        f.write(encoded_data)

    print(f"Encoded data saved to {output_file}")
    return len(indices), len(padded_indices), encoded_data

# Generate simulated data
original_size = 12288  # Original data size (indices count)
indices = np.random.randint(0, 256, size=(original_size,), dtype=np.uint8)  # uint8 indices

# Transmit the encoded data
original_size, padded_size, encoded_data = transmitter(indices, "transmitted_data.bin")




def add_noise(input_file, output_file, error_ratio):
    """
    Add noise to the data in a binary file based on a given error ratio.
    
    Args:
        input_file (str): Path to the input binary file.
        output_file (str): Path to save the noisy binary file.
        error_ratio (float): Fraction of bits to flip (e.g., 0.01 for 1% of bits).
    
    Returns:
        noisy_data (bytes): The noisy data.
    """
    # Read encoded data from file
    with open(input_file, "rb") as f:
        encoded_data = f.read()

    # Calculate the total number of bits in the file
    total_bits = len(encoded_data) * 8

    # Calculate the number of errors based on the error ratio
    num_errors = int(total_bits * error_ratio)
    print(f"Introducing {num_errors} errors out of {total_bits} total bits (ratio = {error_ratio}).")

    # Add noise to the data
    noisy_data = bytearray(encoded_data)
    for _ in range(num_errors):
        byte_index = np.random.randint(len(noisy_data))
        bit_index = np.random.randint(8)
        noisy_data[byte_index] ^= (1 << bit_index)  # Flip a random bit

    # Save the noisy data to a .bin file
    with open(output_file, "wb") as f:
        f.write(noisy_data)

    print(f"Noisy data saved to {output_file}")
    return noisy_data



def receiver(input_file, output_file, original_size):
    # Load noisy data
    with open(input_file, "rb") as f:
        received_data = f.read()

    # Decode block by block
    def decode_blocks(encoded_data, block_size, data_size, redundancy):
        decoded_result = bytearray()
        num_blocks = len(encoded_data) // block_size

        print(f"Decoding {num_blocks} blocks of size {block_size} bytes each.")

        for i in range(num_blocks):
            start = i * block_size
            end = start + block_size
            block = encoded_data[start:end]

            try:
                # Attempt to decode the block
                decoded_block = rs.decode(block)
                if isinstance(decoded_block, tuple):  # Handle tuple output
                    decoded_block = decoded_block[0]
                decoded_result.extend(decoded_block[:data_size])  # Append only data symbols
            except reedsolo.ReedSolomonError:
                # Decoding failed, append raw noisy data for this block
                print(f"Block {i + 1} could not be decoded. Retaining noisy data.")
                decoded_result.extend(block[:data_size])  # Append raw data symbols (exclude redundancy)

        return decoded_result

    # Decode the noisy data
    decoded_data = decode_blocks(received_data, block_size=n, data_size=k, redundancy=r)

    # Remove padding
    decoded_data = decoded_data[:original_size]  # Trim to original data size (bytes)
    decoded_indices = np.frombuffer(decoded_data, dtype=np.uint8)

    # Save decoded indices to a .bin file
    with open(output_file, "wb") as f:
        f.write(decoded_indices.tobytes())

    print(f"Decoded indices saved to {output_file}")
    return decoded_indices

# Transmit the data
original_size, padded_size, encoded_data = transmitter(indices, "transmitted_data.bin")

# Simulate noise
#noisy_data = add_noise("transmitted_data.bin", "noisy_data.bin", num_errors=30)
noisy_data = add_noise("transmitted_data.bin", "noisy_data.bin", error_ratio= 0.04)


# Receive and decode the data
decoded_indices = receiver("noisy_data.bin", "decoded_indices.bin", original_size)

# Validate shapes
print("\nDecoded Indices (Best Effort):")
print(decoded_indices)

print("\nOriginal Indices:")
print(indices)

print(f"\nOriginal shape: {indices.shape}, Decoded shape: {decoded_indices.shape}")


Encoded data saved to transmitted_data.bin
Encoded data saved to transmitted_data.bin
Introducing 6936 errors out of 173400 total bits (ratio = 0.04).
Noisy data saved to noisy_data.bin
Decoding 85 blocks of size 255 bytes each.
Block 1 could not be decoded. Retaining noisy data.
Block 2 could not be decoded. Retaining noisy data.
Block 3 could not be decoded. Retaining noisy data.
Block 4 could not be decoded. Retaining noisy data.
Block 5 could not be decoded. Retaining noisy data.
Block 6 could not be decoded. Retaining noisy data.
Block 7 could not be decoded. Retaining noisy data.
Block 8 could not be decoded. Retaining noisy data.
Block 9 could not be decoded. Retaining noisy data.
Block 10 could not be decoded. Retaining noisy data.
Block 11 could not be decoded. Retaining noisy data.
Block 12 could not be decoded. Retaining noisy data.
Block 13 could not be decoded. Retaining noisy data.
Block 14 could not be decoded. Retaining noisy data.
Block 15 could not be decoded. Retaini

In [44]:
# calculate number of similar elements between indices and decoded indices and give percentage

def calculate_similarity(indices, decoded_indices):
    num_similar = np.sum(indices == decoded_indices)
    total_elements = indices.size
    similarity = num_similar / total_elements
    return similarity, num_similar, total_elements

similarity,num_similar, total_elements = calculate_similarity(indices, decoded_indices)
print(f"\nSimilarity: {similarity:.2%}")
print(f"Errors: {total_elements - num_similar} out of {total_elements} elements")


Similarity: 74.03%
Errors: 3191 out of 12288 elements


In [45]:
def calculate_ber(bin_file1, bin_file2):
    with open(bin_file1, 'rb') as file1, open(bin_file2, 'rb') as file2:
        data1 = file1.read()
        data2 = file2.read()
    
    # Calculate the number of bit errors
    bit_errors = sum(bin(int(byte1) ^ int(byte2)).count('1') for byte1, byte2 in zip(data1, data2))
    
    # Calculate the bit error rate
    ber = bit_errors / (8 * len(data1))
    return ber, bit_errors, len(data1)*8

ber, errors, bits = calculate_ber('transmitted_data.bin', 'noisy_data.bin')
print(f"Bit Error Rate: {ber:.2e}")
print(f"Number of bit errors: {errors}")
print(f"Total number of bits: {bits}")

Bit Error Rate: 3.85e-02
Number of bit errors: 6678
Total number of bits: 173400


In [ ]:


ber, errors, bits = calculate_ber('encoded_feature_rayleigh_image24.bin', 'corrupted_feature_rayleigh_image24.bin')
print(f"Bit Error Rate: {ber:.2e}")
print(f"Number of bit errors: {errors}")
print(f"Total number of bits: {bits}")